In [1]:
# Cell 1: Install core libraries
%pip install --upgrade git+https://github.com/huggingface/diffusers accelerate transformers safetensors huggingface_hub xformers torch torchvision

  Cloning https://github.com/huggingface/diffusers to /tmp/pip-req-build-_0h_ng07
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/diffusers /tmp/pip-req-build-_0h_ng07
  Resolved https://github.com/huggingface/diffusers to commit bbd0c161b55ba2234304f1e6325832dd69c60565
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.7/354.7 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 95.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.4/43.4 MB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 118.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 93.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 59.4 MB/s eta 0:00:

In [2]:
# Cell 2: Mount Drive for persistent storage
from google.colab import drive
drive.mount('/content/drive')



Mounted at /content/drive


In [3]:
# delete a-me-generator folder if it exists
import os
if os.path.exists('/content/a-me-generator'):
    !rm -rf '/content/a-me-generator'

# Clone the repository
!git clone https://github.com/vanthomiy/a-me-generator.git

# Set the repo root
REPO_ROOT = "/content/a-me-generator"

# Copy the 'content' directory to the Colab working directory
!cp -r "$REPO_ROOT/content" /content/

# Copy the training script to the Colab working directory
!cp "$REPO_ROOT/train_dreambooth_lora_sdxl.py" /content/


Cloning into 'a-me-generator'...
remote: Enumerating objects: 12, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 12 (delta 3), reused 11 (delta 2), pack-reused 0 (from 0)
Receiving objects: 100% (12/12), 21.88 KiB | 487.00 KiB/s, done.
Resolving deltas: 100% (3/3), done.
cp: cannot stat '/content/a-me-generator/content': No such file or directory


In [4]:
# Cell 3: Login to Hugging Face Hub (to pull SDXL and push checkpoints)
from huggingface_hub import notebook_login
notebook_login()

In [5]:
# Cell 4: Define paths
INSTANCE_DIR = "/content/drive/MyDrive/a-me-generator/instance_images"  # 3–5 selfies
CLASS_DIR    = "/content/drive/MyDrive/a-me-generator/class_images"     # optional generic faces
OUTPUT_DIR   = "/content/drive/MyDrive/a-me-generator/output_model"
GENERATED_IMAGE_DIR = "/content/drive/MyDrive/a-me-generator/generated_images"

# (Optional) Download or sync your images into INSTANCE_DIR before running.
dirs = [INSTANCE_DIR, CLASS_DIR]

for dir in dirs:
    if not os.path.exists(dir):
        os.makedirs(dir)
        print(f"Directory {dir} created.")
    else:
        print(f"Directory {dir} already exists.")

    # check if the directory is empty
    if not os.listdir(dir):
        print(f"Directory {dir} is empty.")
        from google.colab import files
        uploaded = files.upload()
        for filename in uploaded.keys():
            print(f'User uploaded file "{filename}" with length {len(uploaded[filename])} bytes')

            # save the file to the instance directory
            with open(os.path.join(dir, filename), 'wb') as f:
                f.write(uploaded[filename])

    else:
        print(f"Directory {dir} is not empty.")

Directory /content/drive/MyDrive/a-me-generator/instance_images already exists.
Directory /content/drive/MyDrive/a-me-generator/instance_images is not empty.
Directory /content/drive/MyDrive/a-me-generator/class_images already exists.
Directory /content/drive/MyDrive/a-me-generator/class_images is empty.


In [5]:
MODEL_TO_USE = 'stabilityai/stable-diffusion-xl-base-1.0'

In [6]:
# Cell 5: Launch training
!accelerate launch a-me-generator/train_dreambooth_lora_sdxl.py \
  --pretrained_model_name_or_path=$MODEL_TO_USE \
  --instance_data_dir=$INSTANCE_DIR \
  --class_data_dir=$CLASS_DIR \
  --instance_prompt="a photograph of my_face" \
  --class_prompt="a photograph of a person" \
  --resolution=512 \
  --train_batch_size=1 \
  --gradient_accumulation_steps=4 \
  --learning_rate=1e-4 \
  --max_train_steps=800 \
  --mixed_precision="fp16" \
  --enable_xformers_memory_efficient_attention \
  --output_dir=$OUTPUT_DIR


The following values were not passed to `accelerate launch` and had defaults used instead:
	`--num_processes` was set to a value of `1`
	`--num_machines` was set to a value of `1`
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
2025-04-18 15:20:19.384403: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744989619.616601    1792 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744989619.678703    1792 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-18 15:20:20.186468: I tensorflow/core/platform

In [8]:
from diffusers import StableDiffusionXLPipeline
import torch

# 1. Choose device
device = "cuda" if torch.cuda.is_available() else "cpu"
dtype = torch.float16 if device == "cuda" else torch.float32

# 2. Load base SDXL model
pipe = StableDiffusionXLPipeline.from_pretrained(
    MODEL_TO_USE,
    torch_dtype=dtype,
    use_safetensors=True,
).to(device)

# 3. Load LoRA weights into the base pipeline
pipe.load_lora_weights(OUTPUT_DIR, adapter_name="default")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

No LoRA keys associated to CLIPTextModel found with the prefix='text_encoder'. This is safe to ignore if LoRA state dict didn't originally have any CLIPTextModel related params. You can also try specifying `prefix=None` to resolve the warning. Otherwise, open an issue if you think it's unexpected: https://github.com/huggingface/diffusers/issues/new
No LoRA keys associated to CLIPTextModelWithProjection found with the prefix='text_encoder_2'. This is safe to ignore if LoRA state dict didn't originally have any CLIPTextModelWithProjection related params. You can also try specifying `prefix=None` to resolve the warning. Otherwise, open an issue if you think it's unexpected: https://github.com/huggingface/diffusers/issues/new


In [10]:
# Cell 7: Text-to-image generation
prompt = "A high‑resolution portrait of my_face standing in a sunlit garden, ultra realistic"
image = pipe(prompt, num_inference_steps=50, guidance_scale=7.5).images[0]
image.save(GENERATED_IMAGE_DIR + "/generated_portrait.png")
image.show()

  0%|          | 0/50 [00:00<?, ?it/s]

In [13]:
from diffusers import StableDiffusionXLImg2ImgPipeline
from PIL import Image
import torch

# Setup
device = "cuda" if torch.cuda.is_available() else "cpu"
dtype = torch.float16 if device == "cuda" else torch.float32

# Load base model
pipe = StableDiffusionXLImg2ImgPipeline.from_pretrained(
    MODEL_TO_USE,
    torch_dtype=dtype,
    use_safetensors=True,
).to(device)

# Load your LoRA
pipe.load_lora_weights(OUTPUT_DIR, adapter_name="default")


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

No LoRA keys associated to CLIPTextModel found with the prefix='text_encoder'. This is safe to ignore if LoRA state dict didn't originally have any CLIPTextModel related params. You can also try specifying `prefix=None` to resolve the warning. Otherwise, open an issue if you think it's unexpected: https://github.com/huggingface/diffusers/issues/new
No LoRA keys associated to CLIPTextModelWithProjection found with the prefix='text_encoder_2'. This is safe to ignore if LoRA state dict didn't originally have any CLIPTextModelWithProjection related params. You can also try specifying `prefix=None` to resolve the warning. Otherwise, open an issue if you think it's unexpected: https://github.com/huggingface/diffusers/issues/new


In [15]:
import io

# 1. Upload image
uploaded = files.upload()
input_image = list(uploaded.values())[0]
image = Image.open(io.BytesIO(input_image)).convert("RGB").resize((1024, 1024))

# 5. Prompt & generate
prompt = "Change the t-shirt color to red"
output = pipe(prompt=prompt, image=image, strength=0.75, num_inference_steps=50, guidance_scale=7.5)
edited_image = output.images[0]

# 6. Save & show
save_path = GENERATED_IMAGE_DIR + "/generated_portrait_ii.png"
edited_image.save(save_path)
edited_image.show()

Saving 20230526_160109 1.jpg to 20230526_160109 1 (1).jpg


  0%|          | 0/37 [00:00<?, ?it/s]